In [16]:
import pandas as pd
import glob
import geopandas as gpd
import hdbscan
import matplotlib.pyplot as plt
from pyproj import Transformer
import folium
import re


### Samenvoegen van alle meetputen 

In [4]:
# Alle meetpunten inladen vanuit subfolders van Meetpunten
csv_files = glob.glob('Meetpunten/*/*.csv')  # zoekt in alle subfolders

# DataFrames inlezen en samenvoegen
df_list = []
for file in csv_files:
    df = pd.read_csv(file, sep=';', encoding='utf-8')  # gebruik ';' als delimiter
    df_list.append(df)

# Samenvoegen in één DataFrame
df_all = pd.concat(df_list, ignore_index=True)
print(f"Totaal aantal meetpunten: {len(df_all)}")

# Exporteren naar CSV
df_all.to_csv('all_meetpunten.csv', index=False)
print("Alle meetpunten opgeslagen in 'all_meetpunten.csv'")


Totaal aantal meetpunten: 3342
Alle meetpunten opgeslagen in 'all_meetpunten.csv'


### Clusters maken

In [7]:
# 2. Clustering met HDBSCAN op RD-coördinaten (meters)
# Zorg dat de kolommen 'GeometriePuntX_RD' en 'GeometriePuntY_RD' bestaan
df_all = df_all.dropna(subset=['GeometriePuntX_RD', 'GeometriePuntY_RD'])
coords = df_all[['GeometriePuntX_RD', 'GeometriePuntY_RD']].values

# HDBSCAN-instantie: pas min_cluster_size en min_samples aan naar behoefte
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=10,
    min_samples=1,
    metric='euclidean'
)
labels = clusterer.fit_predict(coords)

# 3. Resultaten toevoegen aan DataFrame
df_all['cluster_id'] = labels
# Optioneel: waarschijnlijkheidscore per punt
df_all['cluster_prob'] = clusterer.probabilities_

# 4. Opslaan van de CSV met clusters
df_all.to_csv('meetpunten_met_clusters.csv', index=False)
print("CSV 'meetpunten_met_clusters.csv' opgeslagen met cluster labels.")


CSV 'meetpunten_met_clusters.csv' opgeslagen met cluster labels.


### HTML kaartje maken 

In [ ]:
# 1. Inladen van de CSV met clusters
df_all = pd.read_csv('meetpunten_met_clusters.csv')

# 2. Omzetten van RD-coördinaten naar WGS84 (lon, lat)
transformer = Transformer.from_crs('EPSG:28992', 'EPSG:4326', always_xy=True)
df_all['lon'], df_all['lat'] = transformer.transform(
    df_all['GeometriePuntX_RD'].values,
    df_all['GeometriePuntY_RD'].values
)

# 3. Kleuren toewijzen per cluster_id
def get_cluster_colors(cluster_ids):
    # Voorbeeld kleurenpalet
    palette = [
        'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
        'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple',
        'pink', 'lightblue', 'lightgreen', 'gray'
    ]
    return {cid: palette[i % len(palette)] for i, cid in enumerate(cluster_ids)}

cluster_ids = sorted(df_all['cluster_id'].unique())
color_map = get_cluster_colors(cluster_ids)

# 4. Simpele Folium-kaart maken
def create_cluster_map(df, color_map, zoom_start=10):
    center = [df['lat'].mean(), df['lon'].mean()]
    m = folium.Map(location=center, zoom_start=zoom_start)
    for _, row in df.iterrows():
        color = color_map.get(row['cluster_id'], 'gray')
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=4,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=f"Cluster: {row['cluster_id']}"
        ).add_to(m)
    return m

# 5. Maak kaart en sla op als HTML
m = create_cluster_map(df_all, color_map)
output_file = 'clusters_map_simple.html'
m.save(output_file)
print(f"Eenvoudige interactieve kaart met gekleurde clusters opgeslagen als '{output_file}'")


Eenvoudige interactieve kaart met gekleurde clusters opgeslagen als 'clusters_map_simple.html'


### Relevantie van clusters

In [15]:
# 1. Inladen van bestaand CSV-bestand met cluster-id's
df = pd.read_csv('meetpunten_met_clusters.csv')

# 2. Bepaal de set van alle jaren in de dataset
all_years = set(df['Meetjaar'].dropna().unique())

# 3. Filter noise-cluster (cluster_id = -1) uit voor relevantiebepaling
df_non_noise = df[df['cluster_id'] != -1]

# 4. Groepeer per cluster en bepaal unieke jaren per cluster
cluster_years = (
    df_non_noise.groupby('cluster_id')['Meetjaar']
    .apply(lambda yrs: set(yrs.dropna().unique()))
)

# 5. Identificeer relevante clusters (die elk jaar in all_years voorkomen)
relevant_clusters = [
    cid for cid, yrs in cluster_years.items() if yrs == all_years
]
print(f"Relevante clusters: {relevant_clusters}")

# 6. Voeg een nieuwe kolom 'relevant' toe aan de DataFrame
#    True als het meetpunt behoort tot een relevante cluster, anders False
df['relevant'] = df['cluster_id'].isin(relevant_clusters)

# 7. Opslaan van het nieuwe CSV-bestand
df.to_csv('meetpunten_met_clusters_relevant.csv', index=False)
print("meetpunten_met_clusters_relevant.csv opgeslagen met relevante kolom.")


Relevante clusters: [3, 5, 19, 25, 26, 32, 33, 44, 62, 64, 73, 74, 87, 88, 89, 91, 95, 96, 108, 113, 114, 124]
meetpunten_met_clusters_relevant.csv opgeslagen met relevante kolom.


In [ ]:
Meetwaarden/Waterschap Brabantse Delta/WKP_Meetwaarden_Waterschap Brabantse Delta_*.csv

### Samenvoegen meetwaarden en meetpunten clusters

In [27]:
# -------------------------
# 1. Inladen van meetpunten met clusters en relevantie
# -------------------------
df_clusters = pd.read_csv('meetpunten_met_clusters_relevant.csv')
# Unieke mapping per MeetobjectCode
# We hebben per meetobject één cluster_id en relevant-flag nodig
df_clusters_unique = (
    df_clusters[['MeetobjectCode', 'cluster_id', 'relevant']]
    .drop_duplicates(subset=['MeetobjectCode'])
)

# -------------------------
# 2. Inladen en samenvoegen van meetwaarden (2016–2024)
# -------------------------
meetwaarden_files = glob.glob('Meetwaarden/Waterschap Brabantse Delta/WKP_Meetwaarden_Waterschap Brabantse Delta_*.csv')
waarden_list = []
for file in meetwaarden_files:
    # Jaar uit bestandsnaam halen (bv. '_2016_')
    match = re.search(r'_(\d{4})_', file)
    jaar = int(match.group(1)) if match else None
    df = pd.read_csv(file, sep=';', encoding='utf-8')
    df['year'] = jaar
    waarden_list.append(df)
# Samenvoegen van alle meetwaarden
df_waarden = pd.concat(waarden_list, ignore_index=True)
print(f"Totaal aantal meetwaardenrijen: {len(df_waarden)}")

# -------------------------
# 3. Merge meetwaarden met cluster-info op MeetobjectCode
# -------------------------
merged = df_waarden.merge(
    df_clusters_unique,
    on='MeetobjectCode',
    how='left'
)
print(f"Aantal rijen na merge: {len(merged)}")

# -------------------------
# 4. Bepalen van relevante stoffen per cluster
# -------------------------
# Verwijder noise-cluster (cluster_id = -1)
df_valid = merged[merged['cluster_id'] != -1]
# Set van alle jaren in de data
total_years = set(df_valid['year'].dropna().unique())
print(f"Jaren in dataset: {sorted(total_years)}")

# Groepeer per cluster en stof en verzamel gemeten jaren
grouped = (
    df_valid
    .groupby(['cluster_id', 'ParameterOmschrijving'])['year']
    .apply(lambda yrs: set(yrs.dropna().unique()))
    .reset_index(name='years_measured')
)
# Markeer stoffen die in elk jaar van hun cluster zijn gemeten
grouped['relevant_stof'] = grouped['years_measured'].apply(
    lambda s: s == total_years
)

# Alleen relevante stoffen per cluster
grouped_relevant = grouped[grouped['relevant_stof']]
print(f"Aantal relevante cluster-stof combinaties: {len(grouped_relevant)}")

# -------------------------
# 5. Bepalen van globale relevantie over alle clusters
# -------------------------
# Tel per stof in hoeveel verschillende clusters hij relevant is
# en bepaal stoffen met de hoogste cluster-frequentie
df_counts = (
    grouped_relevant
    .groupby('ParameterOmschrijving')['cluster_id']
    .nunique()
    .reset_index(name='count_clusters')
)
# Sorteren op aflopend aantal clusters
df_counts = df_counts.sort_values('count_clusters', ascending=False)
print("Top 10 stoffen met hoogste cluster frequentie:")
print(df_counts.head(10))

# Optioneel: bepaal aantal clusters
total_clusters = df_valid['cluster_id'].nunique()
print(f"Totaal aantal unieke clusters (excl. noise): {total_clusters}")

# Stoffen die in alle clusters voorkomen
top_all = df_counts[df_counts['count_clusters'] == total_clusters]
print(f"Stoffen in alle {total_clusters} clusters: {top_all['ParameterOmschrijving'].tolist()}")



/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_11399/86638576.py:21: DtypeWarning: Columns (4,7,8,9,15,16,17,25,26,34,35,40,41,42,43,48,49,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_11399/86638576.py:21: DtypeWarning: Columns (4,7,8,9,16,17,22,34,35,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_11399/86638576.py:21: DtypeWarning: Columns (4,7,8,20,24,25,26,34,35,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=';', encoding='utf-8')
/var/folders/mg/rgpdn71s30sccy9pj35kkd6c0000gn/T/ipykernel_11399/86638576.py:21: DtypeWarning: Columns (4,7,8,20,24,25,26,34,35,42,43) have mixed types. Specify dtype option on import or set l

Totaal aantal meetwaardenrijen: 1115673
Aantal rijen na merge: 1115673
Jaren in dataset: [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Aantal relevante cluster-stof combinaties: 1785
Top 10 stoffen met hoogste cluster frequentie:
    ParameterOmschrijving  count_clusters
180              zuurstof              48
5                  Schuim              47
4                    Olie              42
6                    Vuil              39
40            chlorofyl-a              38
15               ammoniak              33
132               nitraat              33
133               nitriet              33
88          fosfor totaal              33
86                fosfaat              33
Totaal aantal unieke clusters (excl. noise): 87
Stoffen in alle 87 clusters: []
